# Lista 8 (4pkt.+5pkt.) Termin: 14.01.2020r.

In [1]:
import numpy as np
import random
import hashlib

## Zadanie 1 (2 pkt.)

Alicja i Bob ustalają publicznie grupę $Z_p$ w której będą działać oraz element z tej grupy $g$.

In [2]:
g=5
p=23

Alicja wybiera losowy element $a$ grupy $Z_p$, taki, że $1<a<p$:

In [3]:
a=random.randint(2, p-1)

Następnie oblicza $A=g^a$ w grupie $Z_p$, czyli $A=g^a\mod\ p$:

In [4]:
def pow_mod(x,n,m):
    p = 1
    while n:
        if n & 1:
            p = p * x % m
        n >>= 1
        x = x * x % m
    return p

A= pow_mod(g,a,p)
print('A=', A)

A= 9


To samo robi Bob, wybiera losowy elemnt grupy $b$ i oblicza $B=g^b$:

In [5]:
b=random.randint(2, p-1)
B=pow_mod(g,b,p)
print('B=', B)

B= 22


Elementy $a$ i $b$ są znane tylko, odpowiednio, Alicji i Bobowi, natomiast $A$ oraz $B$ są wymieniane publicznym kanałem. Bob otrzymuje $A$ i wykonuje operację $k_B=A^b$ (czyli $A^b\mod\ p$), z kolei Alicja otrzymuje $B$ i wykonuje operację $k_A=B^a$ (czyli $A^b\mod\ p$):

In [6]:
k_B=pow_mod(A,b,p)
# print('k_B=', k_B)
k_A=pow_mod(B,a,p)

print( k_A == k_B )

True


Zawuażmy, że $k_A=k_B$, wynika to z tego, że $k_A=B^a=(g^b)^a=g^{ab}$ oraz $k_B=A^b=(g^a)^b=g^{ba}$ a oczywiście $ab=ba$. Zatem Alicja i Bob dysponują tą samą liczbą, którą mogą użyć jako klucz do szyfrowania. Publicznie znane są wartości $g$, $p$, $g^a$ oraz $g^b$, jednak odzyskanie z nich $a$ i $b$ jest bardzo trudne przy odpwiednim doborze grupy $Z_p$.

## Zadanie 2 (2 pkt.)

Wczytaj dwa obrazki: 'Campin_Boze_Narodzenie1.png' oraz 'Campin_Boze_Narodzenie2.png' jeden z nich jest zgodny z oryginałem obrazu Roberta Campin a drugi jest lekko zmodyfikowany, rozponaj prawdziwy obraz obliczając jego funkcję skrótu SHA-256 i porównując ją do skrótu podanego poniżej (w postaci szesnastkowej), który odpowiada oryginałowi:

In [7]:
hash_original='2d08246bbb3a8f6b0c16b8d8effd2c783233a6a174496731dfff887bb41bc7e7'

Obrazek wczytaj za pomocą:

In [8]:
file = open('Campin_Boze_Narodzenie1.png','rb')
obraz1=file.read()
readable_hash1 = hashlib.sha256(obraz1).hexdigest();
# print("hash1=", readable_hash1)
file.close()

file = open('Campin_Boze_Narodzenie2.png','rb')
obraz2=file.read()
readable_hash2 = hashlib.sha256(obraz2).hexdigest();
# print("hash2=", readable_hash2)
file.close()

print("obraz1 - original = ", readable_hash1 == hash_original)
print("obraz2 - original = ", readable_hash2 == hash_original)

obraz1 - original =  True
obraz2 - original =  False


'rb' w powyższej funkcji oznacza, że odczytujemy binarną postać obrazu. Podobnie wczytaj drugi obraz. Do policzenia funkcji skrótu użyj biblioteki hashlib: https://docs.python.org/3/library/hashlib.html

In [9]:
import hashlib


## Zadanie 3 * (3 pkt.)

Na poprzednich listach pojawiała się funckja przesuwająca zadany ciag bitów o daną liczbę miejsc w lewo, np.:

In [10]:
def shift_left(tab,n):
    return tab[n:]+tab[:n]

In [11]:
shift_left('11101',2)

'10111'

Funkcja ta jednak działa na ciągu znaków tekstowych a nie bitach, napisz tą funkcję jeszcze raz tak aby działała na int-ach reprezentujących ciagi bitów:

In [3]:
def dec2bin(dec_str,pad):
    n = int(dec_str)
    bina = ''
    while n > 0:
        bina = str(n%2) + bina
        n = n//2
    if len(bina)<pad:
        left = pad-len(bina)
        bina = left*"0" + bina
    return bina


def bin2dec(bin_str):
    l = len(bin_str)
    dec = 0 
    for i in range(0, l):
        dec = dec + int(bin_str[i])*(2**(l-i-1))
    return dec



def Shift_left(k,n,n_bits):
    res = k<<n
    print("res=",res)
    a = dec2bin(res, n_bits)
    print("a=",a)
    if len(a) > n_bits:
        over=len(a)-n_bits
        c=a[over:n_bits]+a[:over]
        return bin2dec(c)
    return res 


tzn. powyższy przykład wyglądałby następująco:

In [4]:
print(Shift_left(29,2,5)==23)

res= 116
a= 1110100
True


bo '11101'=29 oraz '10111'=23, natomiast n_bits=5, oznacza, że ciąg bitów ma dlugość 5. W implementacji użyj operatorów takich jak: <<, |, >>, & (https://www.geeksforgeeks.org/python-bitwise-operators/)

## Zadanie 4 * (2 pkt.)

Na wykładzie byla pokazana metoda szybszej deszfyracji w RSA (zmiejszenia wykładnika w potęgowaniu):
$$M=(C^{d_q}+hq)\mod n$$
na wykładzie był podany wzór bez $\mod n$, jednak jest ono potrzebne, n=pq, pozostale oznaczenia jak na wykładzie (notatki na Teamsach), zaimplementuj powyższy wzór.

In [14]:
p = 0x19fbd41d69aa3d86009a967db3379c63cd501f24f7
q = 0x1b6f141f98eeb619bc0360220160a5f75ea07cdf1d
e = 65537

def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)
    

def inv(p,n):
    g, x, y = egcd(p, n)
    if g != 1:
        raise Exception('odwrotność modularna nie istnieje')
    else:
        return x % n


    
dp = inv(e, (p-1)) 
dq = inv(e, (q-1)) 
qinv = inv(q, p)

def decrypt(c):
    m1 = pow(c, dp, p)
    m2 = pow(c, dq, q)
    h = (qinv * (m1 - m2)) % p 
    m = m2 + h * q
    return dec2bin(m, 0)

